In [ ]:
import os

# Add root directory to python path
import sys
from tqdm import tqdm
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src.cmesrc.utils import filepaths_updated_swan_data

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
SWAN = filepaths_updated_swan_data()

In [ ]:
import pandas as pd 

for harpnum, filepath in tqdm(SWAN.items()):

    data = pd.read_csv(filepath, sep="\t")

    plt.close()
    plt.clf()

    fig, ax = plt.subplots(2, figsize=(10,30), num=1, clear=True, sharex=True)

    X = np.arange(len(data), dtype=np.float64)
    lon_min = data["LONDTMIN"].values
    lon_max = data["LONDTMAX"].values
    lat_min = data["LATDTMIN"].values
    lat_max = data["LATDTMAX"].values

    rotated_mask = data["IRBB"].values

    original_x = np.copy(X)
    original_lon_min = np.copy(lon_min)
    original_lon_max = np.copy(lon_max)
    original_lat_min = np.copy(lat_min)
    original_lat_max = np.copy(lat_max)

    original_x[rotated_mask] = np.nan
    original_lon_min[rotated_mask] = np.nan
    original_lon_max[rotated_mask] = np.nan
    original_lat_min[rotated_mask] = np.nan
    original_lat_max[rotated_mask] = np.nan

    rotated_x = np.copy(X)
    rotated_lon_min = np.copy(lon_min)
    rotated_lon_max = np.copy(lon_max)
    rotated_lat_min = np.copy(lat_min)
    rotated_lat_max = np.copy(lat_max)

    rotated_x[~rotated_mask] = np.nan
    rotated_lon_min[~rotated_mask] = np.nan
    rotated_lon_max[~rotated_mask] = np.nan
    rotated_lat_min[~rotated_mask] = np.nan
    rotated_lat_max[~rotated_mask] = np.nan

    ax[0].plot(original_x, original_lon_min, color="black", linestyle="solid")
    ax[0].plot(original_x, original_lon_max, color="orange", linestyle="solid")

    ax[0].plot(rotated_x, rotated_lon_min, color="black", linestyle="dotted")
    ax[0].plot(rotated_x, rotated_lon_max, color="orange", linestyle="dotted")

    ax[0].set_title(f"HARP {harpnum} LON")

    ax[1].plot(original_x, original_lat_min, color="black", linestyle="solid")
    ax[1].plot(original_x, original_lat_max, color="orange", linestyle="solid")

    ax[1].plot(rotated_x, rotated_lat_min, color="black", linestyle="dotted")
    ax[1].plot(rotated_x, rotated_lat_max, color="orange", linestyle="dotted")

    ax[1].set_title(f"HARP {harpnum} LAT")

    plt.savefig(f"figures/bboxes/{harpnum}.png", dpi=150)

    plt.clf()
    plt.cla()
    plt.close()

    fig.clear()
    plt.close(fig)


In [ ]:
import drms

client = drms.Client(email="ucapern@ucl.ac.uk", verbose=True)
harpnum = 12
hmi_request = f"hmi.sharp_720s[{harpnum}]"

keywords = client.query(hmi_request, key="LAT_MIN, LON_MIN, LAT_MAX, LON_MAX, LONDTMIN, LONDTMAX, LATDTMIN, LATDTMAX, T_REC")

In [ ]:
from astropy.time import Time

Time(keywords["T_REC"].apply(lambda x: x[:-4].replace(".","-").replace("_"," ")).to_list())[0]

In [ ]:
ORIGINAL_SWAN[2].iloc[0].name

In [ ]:
keywords

LON_MIN = keywords["LON_MIN"].values
LON_MAX = keywords["LON_MAX"].values
LAT_MIN = keywords["LAT_MIN"].values
LAT_MAX = keywords["LAT_MAX"].values

LONDTMIN = keywords["LONDTMIN"].values
LONDTMAX = keywords["LONDTMAX"].values
LATDTMIN = keywords["LATDTMIN"].values
LATDTMAX = keywords["LATDTMAX"].values


In [ ]:
from matplotlib import animation
import matplotlib.patches as patches

fig, ax = plt.subplots()

dtbox = patches.Rectangle(
    (LONDTMIN[0], LATDTMIN[0]),
    LONDTMAX[0] - LONDTMIN[0],
    LATDTMAX[0] - LATDTMIN[0],
    fc="forestgreen",
    zorder=10
)

box = patches.Rectangle(
    (LON_MIN[0], LAT_MIN[0]),
    LON_MAX[0] - LON_MIN[0],
    LAT_MAX[0] - LAT_MIN[0],
    fc="red",
    zorder=20
)

def init():
    ax.add_patch(dtbox)
    ax.add_patch(box)
    return dtbox, box

def animate(i):
    print([LONDTMIN[i], LATDTMIN[i]])
    dtbox.set_xy([LONDTMIN[i], LATDTMIN[i]])
    dtbox.set_width(LONDTMAX[i] - LONDTMIN[i])
    dtbox.set_height(LATDTMAX[i] - LATDTMIN[i])

    box.set_xy([LON_MIN[i], LAT_MIN[i]])
    box.set_width(LON_MAX[i] - LON_MIN[i])
    box.set_height(LAT_MAX[i] - LAT_MIN[i])
    return dtbox, box

ax.set_xlim(-90,90)
ax.set_ylim(-90,90)

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(LON_MIN), interval=100, blit=True)

writer = animation.FFMpegWriter(fps=60)

anim.save("figures/bboxes/animation.mp4", writer=writer, dpi=200)

In [ ]:
from src.cmesrc.utils import cache_swan_data
from src.cmesrc.config import UPDATED_SWAN
import pandas as pd
from astropy.time import Time
from os.path import join
from tqdm import tqdm
import drms

client = drms.Client(email="ucapern@ucl.ac.uk", verbose=True)

SWAN = cache_swan_data()

In [ ]:
harpnum = 7302

data = SWAN[7302].copy()

df = pd.read_csv(f"/home/julio/cmesrc/data/raw/mvts/SWAN/partition5/{harpnum}.csv", sep="\t")

hmi_request = f"hmi.sharp_720s[{harpnum}]"

keywords = client.query(hmi_request, key="LONDTMIN, LONDTMAX, LATDTMIN, LATDTMAX, T_REC")

In [ ]:
data

In [ ]:
keywords["Timestamp"] = Time(keywords["T_REC"].apply(lambda x: x[:-4].replace(".","-").replace("_"," ")).to_list())
keywords.drop(columns=["T_REC"], inplace=True)

In [ ]:
# data.drop(columns=["Timestamp"], inplace=True)

In [ ]:
data = pd.merge(data, keywords, on="Timestamp", how="outer")

In [ ]:
data.to_csv("test.csv", index=False)

In [ ]:
df = pd.read_csv("test.csv")

In [ ]:
df